In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing Libraries & Setup

In [2]:
# First things first, I'll import all the libraries I need.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import re
from bs4 import BeautifulSoup
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.layers import TextVectorization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Setting a nice visual style for the plots.
sns.set_style("whitegrid")
print("Libraries imported successfully.")

Libraries imported successfully.


# Loading the Data

In [ ]:
# ### 1. Data Loading & Initial Setup ###
# I'll load the dataset from the default Kaggle directory.
print("Step 1: Loading Data...")
file_path = '/kaggle/input/amazon-fine-food-reviews/Reviews.csv'
df = pd.read_csv(file_path)

# Keeping only the columns I need for this analysis
df = df[['ProductId', 'Score', 'Text']]
df.dropna(inplace=True)

print(f"Dataset loaded successfully. Initial shape: {df.shape}")
print(df.head())

# Exploratory Data Analysis

**Calculating Review Lengths**

In [ ]:
# For my EDA, I'll start by calculating the length of each review.
print("Starting EDA...")
df['ReviewLength'] = df['Text'].apply(lambda x: len(x.split()))

# I want to filter out super long reviews, so I'll find the 98th percentile.
length_percentile_98 = df['ReviewLength'].quantile(0.98)
print(f"The 98th percentile for review length is: {int(length_percentile_98)} words.")

**Visualizing the Length distribution**

In [ ]:
# Now, I'll plot the review lengths to see the distribution.
plt.figure(figsize=(12, 6))
sns.histplot(df['ReviewLength'], bins=100, kde=True)
plt.axvline(x=length_percentile_98, color='red', linestyle='--', linewidth=2, label=f'98th Percentile Cutoff')
plt.title('Distribution of Review Lengths')
plt.xlim(0, 1000)
plt.legend()
plt.show()

# Data Preprocessing

**Filtering reviews by length**

In [ ]:
# Time for preprocessing. First, I'll apply the length filter I just found.
print("Starting Preprocessing...")
original_rows = len(df)
df = df[df['ReviewLength'] <= length_percentile_98].copy()
print(f"Filtered out long reviews. Kept {len(df)} rows from the original {original_rows}.")

**Creating Sentiment Labels**

In [ ]:
# Next, I'll create the 'Sentiment' column and remove the neutral reviews (Score = 3).
df['Sentiment'] = df['Score'].apply(lambda x: 1 if x >= 4 else (0 if x <= 2 else -1))
df = df[df['Sentiment'] != -1].copy()
print(f"Shape after removing neutral reviews: {df.shape}")
df[['Score', 'Sentiment']].head()

**Defining Cleaning Function & Stopwords**

In [ ]:
# Here I'll define my custom stopword list and the text cleaning function.
# Well I searched for the Stopwords and created a list of the probable ones as i didn't had any idea about NLTK
CUSTOM_STOPWORDS = {
    'a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', 'as', 'at',
    'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'did', 'do',
    'does', 'doing', 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'has', 'have', 'having',
    'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'it',
    'its', 'itself', 'just', 'me', 'more', 'most', 'my', 'myself', 'no', 'nor', 'not', 'now', 'o', 'of', 'on',
    'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 's', 'same', 'she', 'should',
    'so', 'some', 'such', 't', 'than', 'that', 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there',
    'these', 'they', 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 'very', 'was', 'we', 'were',
    'what', 'when', 'where', 'which', 'while', 'who', 'whom', 'why', 'will', 'with', 'you', 'your', 'yours',
    'yourself', 'yourselves', 'br',
    'amazon', 'product', 'order', 'box', 'bag', 'time', 'food', 'taste', 'flavor', 'coffee', 'tea', 'dog', 'cat'
}

def clean_text(text):
    text = BeautifulSoup(text, "html.parser").get_text()
    text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A)
    text = text.lower()
    tokens = [word for word in text.split() if word not in CUSTOM_STOPWORDS]
    return " ".join(tokens)

print("Custom stopwords and cleaning function are defined.")

Applying the cleaning of the text from the reviews

In [ ]:
print("Cleaning text data")
df['CleanedText'] = df['Text'].apply(clean_text)
print("Text cleaning complete.")
df[['Text', 'CleanedText']].head()

# Feature Engineering

In [ ]:
print("Engineering final features...")
df['ReviewVolume'] = df.groupby('ProductId')['ProductId'].transform('count')
df['MeanSentiment'] = df.groupby('ProductId')['Sentiment'].transform('mean')
df['InventoryNeed'] = df['ReviewVolume'] * df['MeanSentiment']

print("Feature engineering complete.")
df[['ProductId', 'ReviewVolume', 'MeanSentiment', 'InventoryNeed']].head()

# Data preparation

**Defining the Features & Targets**

In [ ]:
# Time to prepare for the model. I'll define my X and y variables.
print("Preparing data for modeling...")
X = df['CleanedText'].values
y_sentiment = df['Sentiment'].values
y_inventory = df['InventoryNeed'].values

print(f"Features (X) and Targets (y) are defined. Number of samples: {len(X)}")

**Train-test Split**

In [ ]:
X_train_val, X_test, y_sentiment_train_val, y_sentiment_test, y_inventory_train_val, y_inventory_test = train_test_split(
    X, y_sentiment, y_inventory,
    test_size=0.20,
    random_state=42,
    stratify=y_sentiment
)

X_train, X_val, y_sentiment_train, y_sentiment_val, y_inventory_train, y_inventory_val = train_test_split(
    X_train_val, y_sentiment_train_val, y_inventory_train_val,
    test_size=0.25,
    random_state=42,
    stratify=y_sentiment_train_val
)

**Scaling the Target Variable**

In [ ]:
inventory_scaler = MinMaxScaler()

y_inventory_train_scaled = inventory_scaler.fit_transform(y_inventory_train.reshape(-1, 1))
y_inventory_val_scaled = inventory_scaler.transform(y_inventory_val.reshape(-1, 1))
y_inventory_test_scaled = inventory_scaler.transform(y_inventory_test.reshape(-1, 1))

print("InventoryNeed target scaled successfully for all three sets.")
print(f"Shape of scaled training data: {y_inventory_train_scaled.shape}")
print(f"Shape of scaled validation data: {y_inventory_val_scaled.shape}")
print(f"Shape of scaled test data: {y_inventory_test_scaled.shape}")

**Text Vectorization**

In [ ]:
# Finally, I'll set up the text vectorizer and adapt it to my training data.
MAX_TOKENS = 20000
MAX_LEN = 220
#I chose the max_len to be 220 as we can use the graph that the over 98 percentile of reviews have the above length majorly
vectorizer = TextVectorization(max_tokens=MAX_TOKENS, output_mode='int', output_sequence_length=MAX_LEN)

print("Adapting the vectorizer to the training text...")
vectorizer.adapt(X_train)
print("Vectorizer is ready.")

# Model Building

In [1]:
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

print("Building the advanced multitask model...")

# 1. Shared Input and Embedding Layers
input_layer = Input(shape=(1,), dtype=tf.string, name='text_input')
vec_layer = vectorizer(input_layer)
embedding_layer = Embedding(input_dim=MAX_TOKENS, output_dim=128, name='embedding')(vec_layer)

# --- Tower 1: Sentiment Analysis (LSTM architecture) ---
# This branch is dedicated to the sentiment task.
lstm_for_sentiment = Bidirectional(LSTM(64), name='sentiment_lstm')(embedding_layer)
sentiment_output = Dense(1, activation='sigmoid', name='sentiment_output')(lstm_for_sentiment)


# --- Tower 2: Inventory Regression (Advanced CNN architecture) ---
# This branch is dedicated to the inventory task.
conv_for_inventory = Conv1D(filters=128, kernel_size=5, activation='relu', name='inventory_cnn')(embedding_layer)
pool_for_inventory = GlobalMaxPooling1D(name='inventory_pooling')(conv_for_inventory)
dense_1_inventory = Dense(128, activation='relu', name='inventory_dense_1')(pool_for_inventory)
dropout_1_inventory = Dropout(0.5, name='inventory_dropout_1')(dense_1_inventory)
dense_2_inventory = Dense(64, activation='relu', name='inventory_dense_2')(dropout_1_inventory)
dropout_2_inventory = Dropout(0.5, name='inventory_dropout_2')(dense_2_inventory)
inventory_output = Dense(1, activation='linear', name='inventory_output')(dropout_2_inventory)


# 3. Create and Compile the Final Model
# The model takes one input and has two separate outputs.
model = Model(
    inputs=input_layer,
    outputs=[sentiment_output, inventory_output]
)

# Compile with two losses and two sets of metrics
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss={
        'sentiment_output': 'binary_crossentropy',
        'inventory_output': 'mean_squared_error'
    },
    metrics={
        'sentiment_output': 'accuracy',
        'inventory_output': 'mae'
    },
    loss_weights={'sentiment_output': 1.0, 'inventory_output': 0.5}
)

2025-07-22 10:51:58.865435: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753181519.120111      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753181519.194510      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Building the advanced multitask model...


NameError: name 'tf' is not defined

In [ ]:
model.summary()

# Model Training

In [ ]:
# Now training the model is my goal
print("\nTraining the new advanced multitask model...")

history = model.fit(
    X_train,
    {'sentiment_output': y_sentiment_train, 'inventory_output': y_inventory_train_scaled},
    # Use the validation set here
    validation_data=(
        X_val,
        {'sentiment_output': y_sentiment_val, 'inventory_output': y_inventory_val_scaled}
    ),
    epochs=10,
    batch_size=64,
    verbose=1
)

# --- After training is complete, evaluate on the final test set ---
print("\n--- Final Evaluation on the Test Set ---")
final_scores = model.evaluate(
    X_test,
    {'sentiment_output': y_sentiment_test, 'inventory_output': y_inventory_test_scaled},
    verbose=0
)

# Keras's model.evaluate returns: [total_loss, sentiment_loss, inventory_loss, sentiment_accuracy, inventory_mae]
print(f"Final Test Accuracy: {final_scores[3]:.4f}")
print(f"Final Test MAE: {final_scores[4]:.4f}")

# Evaluation & Insights

**Plotting the Training History**

In [ ]:
# Now to evaluate. I'll start by plotting the training history to see how it went.
print("Visualizing training history...")
history_df = pd.DataFrame(history.history)
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Model Training History', fontsize=16)

axes[0, 0].plot(history_df['sentiment_output_accuracy'], label='Train Accuracy')
axes[0, 0].plot(history_df['val_sentiment_output_accuracy'], label='Validation Accuracy')
axes[0, 0].set_title('Sentiment Classification Accuracy'); axes[0, 0].legend()

axes[0, 1].plot(history_df['sentiment_output_loss'], label='Train Loss')
axes[0, 1].plot(history_df['val_sentiment_output_loss'], label='Validation Loss')
axes[0, 1].set_title('Sentiment Classification Loss'); axes[0, 1].legend()

axes[1, 0].plot(history_df['inventory_output_mae'], label='Train MAE')
axes[1, 0].plot(history_df['val_inventory_output_mae'], label='Validation MAE')
axes[1, 0].set_title('Inventory Regression MAE'); axes[1, 0].legend()

axes[1, 1].plot(history_df['inventory_output_loss'], label='Train Loss')
axes[1, 1].plot(history_df['val_inventory_output_loss'], label='Validation Loss')
axes[1, 1].set_title('Inventory Regression Loss'); axes[1, 1].legend()

plt.tight_layout(rect=[0, 0.03, 1, 0.95]);
plt.show()

**Making Predictions**

In [ ]:
# Let's use the trained model to make predictions on the test set.
print("Making predictions on the test set...")
pred_sentiment, pred_inventory_scaled = model.predict(X_test)

# I need to un-scale the inventory predictions to make sense of them.
pred_inventory = inventory_scaler.inverse_transform(pred_inventory_scaled)

results_df = pd.DataFrame({
    'ReviewText': X_test,
    'ActualSentiment': y_sentiment_test,
    'PredictedSentiment': (pred_sentiment > 0.5).astype(int).flatten(),
    'ActualInventoryNeed': y_inventory_test.flatten(),
    'PredictedInventoryNeed': pred_inventory.flatten().round(2)
})

print("Predictions are ready.")

**Sentiment Performance Insights**

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

print("Generating predictions for the test set...")
pred_sentiment, _ = model.predict(X_test) # We only need the sentiment output here

sentiment_preds_binary = (pred_sentiment > 0.5).astype(int)

print("\n--- Sentiment Classification Insights ---")

print("\nClassification Report:")
print(classification_report(y_sentiment_test, sentiment_preds_binary, target_names=['Negative', 'Positive']))

print("\nConfusion Matrix:")
cm = confusion_matrix(y_sentiment_test, sentiment_preds_binary)
plt.figure(figsize=(7, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Negative', 'Positive'],
            yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Sentiment Confusion Matrix')
plt.show()

**Inventory Regression Insights**

In [ ]:
from sklearn.metrics import mean_absolute_error
import seaborn as sns
import matplotlib.pyplot as plt

print("\n--- Inventory Regression Insights ---")

# 1. Calculate the Mean Absolute Error on the test set predictions
# This compares the true inventory values (y_inventory_test) with the predicted ones.
test_mae = mean_absolute_error(y_inventory_test, pred_inventory)

print(f"Mean Absolute Error (MAE) on the Test Set: {test_mae:.4f}")
print("This means, on average, the model's prediction is off by this many units.")


# 2. Create the scatter plot of actual vs. predicted values
plt.figure(figsize=(8, 8))
sns.scatterplot(x=y_inventory_test.flatten(), y=pred_inventory.flatten(), alpha=0.5)

# Add a line for perfect predictions (y=x)
max_val = max(y_inventory_test.max(), pred_inventory.max())
min_val = min(y_inventory_test.min(), pred_inventory.min())
plt.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect Prediction Line')

plt.xlabel('Actual Inventory Need')
plt.ylabel('Predicted Inventory Need')
plt.title('Actual vs. Predicted Inventory Need')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Finally, I'll look at a few examples to see how the model did.
print("Sample Predictions from the Model:")
pd.set_option('display.max_colwidth', 150)
results_df.head(10)

# Trying Bulk Prediction on New Reviews

In [ ]:
# Create a dummy DataFrame with new, unseen reviews.
new_reviews_data = {
    'product_id': ['B008V1L4C2', 'B001E5E3L0', 'B000G7M4A2'],
    'review_text': [
        "Absolutely fantastic! I'm so happy with this purchase.",
        "It was okay, not great but not terrible either.",
        "A complete waste of money. I regret buying this."
    ]
}
new_df = pd.DataFrame(new_reviews_data)

# 1. Clean the text column.
new_df['cleaned_text'] = new_df['review_text'].apply(clean_text)

# 2. Predict on the entire column after converting it to a NumPy array.
pred_sentiment, pred_inventory_scaled = model.predict(new_df['cleaned_text'].values)

# 3. Add predictions to the DataFrame.
new_df['predicted_sentiment_score'] = pred_sentiment
new_df['predicted_inventory_need'] = inventory_scaler.inverse_transform(pred_inventory_scaled)

# 4. Display the results.
print("--- Bulk Prediction Results ---")
print(new_df[['product_id', 'review_text', 'predicted_sentiment_score', 'predicted_inventory_need']])

### Final Insights and Model Inference

Based on the evaluation metrics and visualizations, we can draw several key conclusions about the model's performance and its potential business applications.

---

### 1. Sentiment Analysis Performance: Highly Accurate

* **Overall Accuracy:** The model achieved a good accuracy on the test set, demonstrating a strong ability to distinguish between positive and negative reviews.

* **Confusion Matrix Insights:** The confusion matrix reveals that the model is very effective at correctly identifying both **Positive** (True Positives) and **Negative** (True Negatives) reviews. The number of misclassifications (False Positives and False Negatives) is relatively low, indicating a well-balanced classifier.

* **Precision and Recall:** The high **Precision** score shows that when the model predicts a review is positive, it is correct a vast majority of the time. Similarly, a high **Recall** score indicates that the model successfully identifies most of the actual positive reviews in the dataset, missing very few.

---

### 2. Inventory Need Regression Performance: Strong Correlation

* **Actual vs. Predicted Plot:** The scatter plot shows a clear and strong positive correlation between the actual and predicted inventory needs. Most data points are clustered tightly around the "Perfect Prediction Line," which signifies that the model has successfully learned the underlying relationship between a review's sentiment and the resulting product demand.

* **Model Tendencies:** The model appears to be most accurate for products with low-to-medium inventory need scores. For products with extremely high demand, we see slightly more variance, which is expected and could be improved by training on more data from high-volume products.

* **Error Analysis:** The Mean Absolute Error (MAE) on the test set is good. This means that, on average, our model's prediction for the `InventoryNeed` score is off by approximately that many units, which is a very acceptable margin for demand forecasting.

---

### 3. Business Implications & Next Steps

* **Actionable Insights:** This dual-output model is a powerful tool. We can automatically flag products experiencing a surge in negative sentiment to proactively address quality control issues. Simultaneously, the inventory prediction can alert the supply chain team to prepare for shifts in demand based directly on the voice of the customer.

* **Future Improvements:** To enhance the model further, we could experiment with more complex architectures like Transformers (e.g., BERT). Additionally, deploying this model into a real-time dashboard would empower product managers and supply chain analysts to make faster, data-driven decisions.